# Designer Agent Notebook

This notebook showcases the Designer Agent, the primary interface for code generation.

## Purpose
The Designer Agent is responsible for translating user requirements into quantum circuits. This notebook demonstrates:

1.  **Agent Setup**: Initializing the Designer Agent with RAG capabilities.
2.  **Task Execution**: Sending natural language tasks (e.g., "Create a Teleportation circuit") to the agent.
3.  **Code Production**: Verifying that the agent produces syntactically correct Cirq code based on the input.

## Usage
Use this notebook to interact with the Designer Agent and generate initial circuit implementations.


In [1]:
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.cirq_rag_code_assistant.config import get_config, setup_logging
from src.rag.knowledge_base import KnowledgeBase
from src.rag.retriever import Retriever
from src.rag.generator import Generator
from src.agents.designer import DesignerAgent

# Setup logging
setup_logging()

2025-12-06 18:35:34 | INFO     | src.cirq_rag_code_assistant.config.logging:setup_all:138 | Logging configuration completed


### Initialize RAG Components
We need to initialize the underlying RAG components first:
1. Load the KnowledgeBase and its entries
2. Build or load the vector index
3. Create Retriever and Generator

In [2]:
# Define paths
KNOWLEDGE_BASE_DIR = project_root / "data" / "knowledge_base"
VECTOR_INDEX_PATH = project_root / "data" / "models" / "vector_index"

# Initialize Knowledge Base and load entries
kb = KnowledgeBase(knowledge_base_path=KNOWLEDGE_BASE_DIR)
num_loaded = kb.load_from_directory()
print(f"Loaded {num_loaded} entries from knowledge base")

# Try to load existing index, or build new one
try:
    kb.load_index(VECTOR_INDEX_PATH)
    print(f"Loaded vector index: {kb.vector_store.size()} entries")
except FileNotFoundError:
    print("No existing index found, building new one...")
    kb.index_entries()
    kb.save_index(VECTOR_INDEX_PATH)
    print(f"Built and saved vector index: {kb.vector_store.size()} entries")

# Initialize RAG components (uses Ollama by default)
retriever = Retriever(knowledge_base=kb, use_hybrid_scoring=True)

try:
    generator = Generator(retriever=retriever)
    print("\n✅ RAG components initialized.")
except Exception as e:
    print(f"\n⚠️ Error initializing Generator: {e}")
    print("Make sure Ollama is running locally.")
    generator = None

2025-12-06 18:35:34 | INFO     | config.config_loader:load:93 | ✅ Loaded configuration from D:\University\Uni\Semester 7\Generative AI\Project\Cirq-RAG-Code-Assistant\config\config.json
2025-12-06 18:35:34 | INFO     | src.rag.embeddings:__init__:97 | Loading embedding model: BAAI/bge-base-en-v1.5
2025-12-06 18:35:34 | INFO     | src.rag.embeddings:__init__:98 | Using device: cpu


2025-12-06 18:35:34,225 - sentence_transformers.SentenceTransformer - INFO - SentenceTransformer.py:227 - Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5


2025-12-06 18:35:39 | INFO     | src.rag.embeddings:__init__:106 | ✅ Embedding model loaded successfully
2025-12-06 18:35:39 | INFO     | src.rag.embeddings:__init__:113 | Embedding dimension: 768
2025-12-06 18:35:39 | INFO     | src.rag.vector_store:_init_faiss:139 | Initialized FAISS index
2025-12-06 18:35:39 | INFO     | src.rag.vector_store:__init__:120 | Initialized VectorStore with faiss backend
2025-12-06 18:35:39 | INFO     | src.rag.vector_store:__init__:121 | Embedding dimension: 768
2025-12-06 18:35:39 | INFO     | src.rag.knowledge_base:__init__:100 | Initialized KnowledgeBase with 0 entries
2025-12-06 18:35:39 | INFO     | src.rag.knowledge_base:load_from_jsonl:116 | Loading knowledge base from D:\University\Uni\Semester 7\Generative AI\Project\Cirq-RAG-Code-Assistant\data\knowledge_base\curated_designer_examples.jsonl
2025-12-06 18:35:39 | INFO     | src.rag.knowledge_base:load_from_jsonl:129 | Loaded 102 entries from D:\University\Uni\Semester 7\Generative AI\Project\Cir

Loaded 102 entries from knowledge base
Loaded vector index: 102 entries

✅ RAG components initialized.


### Initialize Designer Agent
The Designer Agent combines retrieval and generation to produce Cirq code.

In [3]:
if generator is not None:
    # Initialize Designer Agent
    designer = DesignerAgent(retriever=retriever, generator=generator)
    print("✅ Designer Agent initialized.")
else:
    designer = None
    print("⚠️ Cannot initialize Designer Agent without Generator.")
    print("Make sure Ollama is running locally.")

2025-12-06 18:35:39 | INFO     | src.agents.base_agent:__init__:78 | Initialized DesignerAgent agent


✅ Designer Agent initialized.


### Generate Circuit
Let's ask the agent to design a circuit.

In [4]:
if designer is not None:
    task = {
        "query": "Create a circuit for Quantum Teleportation",
        "algorithm": "teleportation"
    }

    try:
        result = designer.execute(task)
        
        if result['success']:
            print("✅ Successfully generated circuit!")
            print("\nCode:")
            print("-" * 40)
            print(result['code'])
            print("-" * 40)
            print(f"\nAlgorithm: {result.get('algorithm', 'N/A')}")
            print(f"Context used: {result.get('context_used', 'N/A')} examples")
        else:
            print(f"❌ Failed to generate circuit: {result.get('error')}")
            if 'code' in result:
                print("\nGenerated code (with errors):")
                print(result['code'])
            
    except Exception as e:
        print(f"Error executing task: {e}")
else:
    print("Designer Agent not available.")

2025-12-06 18:35:39 | INFO     | src.rag.generator:generate:192 | Retrieving context for query: Create a circuit for Quantum Teleportation...
2025-12-06 18:35:39 | INFO     | src.rag.generator:generate:213 | Generating code using ollama/cirq-designer-agent
2025-12-06 18:35:50 | INFO     | src.rag.generator:generate:268 | ✅ Code generation completed


✅ Successfully generated circuit!

Code:
----------------------------------------
import cirq

def quantum_teleportation(source, target):
    # Create a qubit for entanglement
    entangled_qubit = cirq.GridQubit(0, 2)
    
    # Define the circuit
    circuit = cirq.Circuit()
    
    # Step 1: Prepare an entangled pair between source and entangled_qubit
    circuit.append(cirq.H(source))
    circuit.append(cirq.CNOT(source, entangled_qubit))
    
    # Step 2: Bell state measurement on the source and target qubits
    circuit.append(cirq.CNOT(target, source))
    circuit.append(cirq.H(target))
    circuit.append(cirq.measure(target, key='target'))
    circuit.append(cirq.measure(source, key='source'))
    
    # Step 3: Conditional operations on the entangled qubit based on measurement results
    circuit.append(cirq.CNOT(entangled_qubit, target))
    circuit.append(cirq.H(entangled_qubit))
    
    return circuit

# Example usage
source = cirq.GridQubit(0, 0)
target = cirq.GridQubit

### Try More Examples
Let's try a few more circuit generation tasks.

In [5]:
if designer is not None:
    test_tasks = [
        {"query": "Create a Bell state circuit", "algorithm": "bell_state"},
        {"query": "Implement Grover's search for 2 qubits", "algorithm": "grover"},
        {"query": "Build a 3-qubit GHZ state", "algorithm": "ghz_state"},
    ]
    
    for i, task in enumerate(test_tasks, 1):
        print(f"\n{'='*60}")
        print(f"Task {i}: {task['query']}")
        print('='*60)
        
        try:
            result = designer.execute(task)
            if result['success']:
                print("✅ Success")
                # Show first 10 lines of code
                code_lines = result['code'].split('\n')[:10]
                print("Code preview:")
                for line in code_lines:
                    print(f"  {line}")
                if len(result['code'].split('\n')) > 10:
                    print("  ...")
            else:
                print(f"❌ Failed: {result.get('error')}")
        except Exception as e:
            print(f"Error: {e}")

2025-12-06 18:35:50 | INFO     | src.rag.generator:generate:192 | Retrieving context for query: Create a Bell state circuit...
2025-12-06 18:35:50 | INFO     | src.rag.generator:generate:213 | Generating code using ollama/cirq-designer-agent



Task 1: Create a Bell state circuit


2025-12-06 18:35:59 | INFO     | src.rag.generator:generate:268 | ✅ Code generation completed
2025-12-06 18:35:59 | INFO     | src.rag.generator:generate:192 | Retrieving context for query: Implement Grover's search for 2 qubits...
2025-12-06 18:35:59 | INFO     | src.rag.generator:generate:213 | Generating code using ollama/cirq-designer-agent


✅ Success
Code preview:
  import cirq
  
  def create_bell_state():
      # Create two qubits
      q0, q1 = cirq.LineQubit.range(2)
      
      # Initialize a circuit
      circuit = cirq.Circuit()
      
      # Apply Hadamard gate to the first qubit to create superposition
  ...

Task 2: Implement Grover's search for 2 qubits


2025-12-06 18:36:12 | INFO     | src.rag.generator:generate:268 | ✅ Code generation completed
2025-12-06 18:36:12 | INFO     | src.rag.generator:generate:192 | Retrieving context for query: Build a 3-qubit GHZ state...
2025-12-06 18:36:12 | INFO     | src.rag.generator:generate:213 | Generating code using ollama/cirq-designer-agent


✅ Success
Code preview:
  import cirq
  
  def oracle(qubits):
      # Oracle for Grover's algorithm that marks the solution |11>
      yield cirq.CNOT(qubits[0], qubits[1])
      yield cirq.Z(qubits[1])
      yield cirq.CNOT(qubits[0], qubits[1])
  
  
  def diffuser(qubits):
  ...

Task 3: Build a 3-qubit GHZ state


2025-12-06 18:36:23 | INFO     | src.rag.generator:generate:268 | ✅ Code generation completed


✅ Success
Code preview:
  import cirq
  
  def create_ghz_state(qubits):
      """
      Creates a GHZ state on the given qubits.
      Args:
          qubits (list of cirq.Qubit): The qubits to use for the GHZ state.
      Returns:
          cirq.Circuit: A circuit that prepares the GHZ state.
      """
  ...
